## Version 2 Notebook
### Completed Steps
  * Used youtube-transcript-api for getting transcripts of youtube videos
  * Used LamaIndex as a wrapper over the LLM for aditionally functionalities
  * By studying transcript of a single video generated a new script

### Further Steps
  * Get transcripts from 5 videos and input it to llm for response
  * Create a Pipline for these steps
  * Create Streamlit app and add functionalities

### Outcome
Streamlit Youtube video script Generator which takes 5 youtube video as reference

In [1]:
!pip install youtube-transcript-api

In [2]:
%pip install llama-index-llms-gemini

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.3.0 which is incompatible.


In [3]:
!pip install -q llama-index google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 17.1 MB/s eta 0:00:00


In [4]:
import re
from youtube_transcript_api import YouTubeTranscriptApi

def extract_id(video_url):
  """Extracts the video ID from a YouTube URL.

  Args:
      video_url: The YouTube URL string.

  Returns:
      The extracted video ID or None if not found.
  """
  id = None
  try:
    id = re.search(r"(?<=v=)([^&]+)", video_url)
  except:
    pass  # Silently ignore the exception for "v=" case
  if id is None:
    try:
      id = re.search(r"(?<=be\/)([^\?]+)", video_url)
    except:
      pass  # Silently ignore the exception for "be/" case
  return id if id else None

url1="https://www.youtube.com/watch?v=YUdiyhiyVVc"
url2="https://youtu.be/4AzpmZ7AjaQ?si=XkRKyHS1vfSAwLCB"

video_id=v=extract_id(url1)[0]

In [5]:
transcript=YouTubeTranscriptApi.get_transcript(v)

In [6]:
type(transcript)

list

In [7]:
%pip install llama-index-llms-gemini

In [8]:
!pip install -q llama-index google-generativeai

In [9]:
from google.colab import userdata
api_key=userdata.get('GOOGLE_API_KEY')

In [12]:
from llama_index.llms.gemini import Gemini

resp = Gemini(api_key=api_key,model_name="models/gemini-1.5-pro")

print(resp.complete("Write a poem"))

The wind whispers secrets through leaves of jade,
A symphony of rustling, light and shade.
Sunlight dances, dappled on the ground,
Where tiny blooms in vibrant hues abound.

A silver stream, a ribbon through the green,
Reflects the sky, a canvas serene.
A hawk circles high, on currents warm,
A silent guardian, safe from every storm.

The air is sweet with scent of pine and earth,
A gentle breeze, a whispered, joyful birth.
The world unfolds, a tapestry so grand,
A masterpiece, painted by nature's hand. 

So breathe it in, this moment, light and free,
And let your worries fade into the sea.
For in this space, where peace and beauty reign,
Your weary soul can find its strength again. 



In [49]:
prompt="""
You are a Youtube Content Creation Subject Matter Expert.You need to create a
youtube video script on the given{context}.\
you will be given  5 youtube video {transcripts} for reference
you will have to create  a youtube video script for a duration of more than 10 miniutes .
you will make sure the script is structured by seperated timestamps and time blocks and a summary of the context in the time block
Make sure to format it in JSON format
Make sure to make the video transcript that is more than 10 minutes long
JSON output should contain timestamp,timeblock,summary and script
"""

In [50]:
from llama_index.core import PromptTemplate

In [51]:
prompt_tmpl = PromptTemplate(prompt)

In [52]:
partial_prompt_tmpl = prompt_tmpl.partial_format(context="Self Help")

In [53]:
partial_prompt_tmpl.kwargs

{'context': 'Self Help'}

In [54]:

fmt_prompt = partial_prompt_tmpl.format(
    transcripts=transcript
)
print(fmt_prompt)


You are a Youtube Content Creation Subject Matter Expert.You need to create a
youtube video script on the givenSelf Help.you will be given  5 youtube video [{'text': 'Transcriber: Daniela Michelle Palao Pineda\nReviewer: Anna Sobota', 'start': 0.0, 'duration': 7.0}, {'text': 'There’s no such thing\nas being born with self-doubt,', 'start': 1.431, 'duration': 2.736}, {'text': 'and I’ll prove it to you.', 'start': 4.167, 'duration': 1.402}, {'text': 'I challenge you to ask\nany friendly neighborhood 12-month-old', 'start': 5.636, 'duration': 2.936}, {'text': 'at what point they would give up\non learning to walk.', 'start': 8.572, 'duration': 2.569}, {'text': 'They’d likely look at you \nwith bewilderment and mild amusement.', 'start': 11.241, 'duration': 3.037}, {'text': 'Okay, they’ll probably look bewildered\nwhatever you say.', 'start': 14.278, 'duration': 2.736}, {'text': 'The typical 12 month old\ninstinctively keeps trying to walk,', 'start': 17.014, 'duration': 3.036}, {'text': 

In [55]:
model=Gemini(api_key=api_key,model_name="models/gemini-1.5-pro")

In [56]:
from llama_index.core.query_pipeline import QueryPipeline

In [57]:
p = QueryPipeline(chain=[prompt_tmpl, model], verbose=True)

In [58]:
processed_transcript="".join([x['text'] for x in transcript])

In [59]:
response=p.run(context="Self Help",transcripts=processed_transcript)

> Running module b59fbbe4-9434-44e3-8d99-59403ddfb5ad with input: 
context: Self Help
transcripts: Transcriber: Daniela Michelle Palao Pineda
Reviewer: Anna SobotaThere’s no such thing
as being born with self-doubt,and I’ll prove it to you.I challenge you to ask
any friendly neighborhood 12-month-o...

> Running module e274e6b6-103e-486f-aafe-b4a1db89afd5 with input: 
messages: 
You are a Youtube Content Creation Subject Matter Expert.You need to create a
youtube video script on the givenSelf Help.you will be given  5 youtube video Transcriber: Daniela Michelle Palao Pineda
...



In [60]:
print(response)

assistant: ```json
[
  {
    "timestamp": "0:00-0:30",
    "timeblock": "Intro",
    "summary": "Introduction to the concept of self-doubt and how it's a learned behavior, not an inherent trait. The speaker challenges the audience to recall the unwavering determination of a 12-month-old learning to walk, highlighting the contrast with adult self-doubt.",
    "script": "Hey everyone, and welcome to today's video. We're diving deep into the world of self-doubt, and I'm here to tell you something you might find surprising: self-doubt isn't something you're born with. It's learned. Think about it. Have you ever seen a baby learning to walk give up? They stumble, they fall, but they get right back up and try again. They don't have the same internal critic that we develop as adults. So, if we learn self-doubt, we can unlearn it too, right?"
  },
  {
    "timestamp": "0:30-2:00",
    "timeblock": "The Three Mind Pits",
    "summary": "Introduction of the three 'mind pits' of self-doubt: Failu

In [61]:
import json

In [80]:
json_file=response.json()

In [81]:
print(json_file)

{"message": {"role": "assistant", "content": "```json\n[\n  {\n    \"timestamp\": \"0:00-0:30\",\n    \"timeblock\": \"Intro\",\n    \"summary\": \"Introduction to the concept of self-doubt and how it's a learned behavior, not an inherent trait. The speaker challenges the audience to recall the unwavering determination of a 12-month-old learning to walk, highlighting the contrast with adult self-doubt.\",\n    \"script\": \"Hey everyone, and welcome to today's video. We're diving deep into the world of self-doubt, and I'm here to tell you something you might find surprising: self-doubt isn't something you're born with. It's learned. Think about it. Have you ever seen a baby learning to walk give up? They stumble, they fall, but they get right back up and try again. They don't have the same internal critic that we develop as adults. So, if we learn self-doubt, we can unlearn it too, right?\"\n  },\n  {\n    \"timestamp\": \"0:30-2:00\",\n    \"timeblock\": \"The Three Mind Pits\",\n    

In [73]:
#files=json.dumps(json_file)

In [83]:
json_dict=json.loads(json_file)

In [88]:
print(json_dict['message']['content'])

```json
[
  {
    "timestamp": "0:00-0:30",
    "timeblock": "Intro",
    "summary": "Introduction to the concept of self-doubt and how it's a learned behavior, not an inherent trait. The speaker challenges the audience to recall the unwavering determination of a 12-month-old learning to walk, highlighting the contrast with adult self-doubt.",
    "script": "Hey everyone, and welcome to today's video. We're diving deep into the world of self-doubt, and I'm here to tell you something you might find surprising: self-doubt isn't something you're born with. It's learned. Think about it. Have you ever seen a baby learning to walk give up? They stumble, they fall, but they get right back up and try again. They don't have the same internal critic that we develop as adults. So, if we learn self-doubt, we can unlearn it too, right?"
  },
  {
    "timestamp": "0:30-2:00",
    "timeblock": "The Three Mind Pits",
    "summary": "Introduction of the three 'mind pits' of self-doubt: Failure to Launc

# Version 2

In [89]:
urls_for_getting_transcript={"SimpliLearn_Bigdata_Visualization":"https://youtu.be/MiiANxRHSv4?si=FhChhEUuttvV0J_E",
 "ted_talk_data_visulaisation":"https://youtu.be/5Zg-C8AAIGg?si=1AQZF0Cu0gkXSAkE",
 "one_minute_video":"https://youtu.be/xekEXM0Vonc?si=rHsFl9wb2lB0dNzf",
 "Evolution of Visulaisation ted talk":"https://youtu.be/sOJet45-kcY?si=Wzri1TU7n08XjqSM"}

In [91]:
urls=list(urls_for_getting_transcript.values())

In [94]:
video_ids=[extract_id(i)[0] for i in urls]

In [96]:
youtube_transcripts=YouTubeTranscriptApi.get_transcripts(video_ids, languages=['en'])

In [104]:
video_ids

['MiiANxRHSv4', '5Zg-C8AAIGg', 'xekEXM0Vonc', 'sOJet45-kcY']

In [103]:
youtube_transcripts[0]['MiiANxRHSv4'][0]

{'text': "let's now start this lesson by defining",
 'start': 10.24,
 'duration': 4.399}

In [108]:
youtube_transcripts[0]['xekEXM0Vonc'][0]

{'text': '[Music]', 'start': 10.12, 'duration': 5.31}

In [110]:
youtube_transcripts[0]['5Zg-C8AAIGg'][0]

{'text': '[Music]', 'start': 2.58, 'duration': 3.16}

In [123]:
unprocessed_transcripts_set=[]
for i in video_ids:
  unprocessed_transcripts_set.append(youtube_transcripts[0][i])

In [128]:
processed_transcripts_set=[]
for the_transcript in unprocessed_transcripts_set:
      processed_transcripts_set.append("".join([x['text'] for x in the_transcript]))

In [129]:
prompt_template=PromptTemplate(prompt)

In [130]:
pipeline = QueryPipeline(chain=[prompt_template, model], verbose=True)

In [131]:
llm_response=pipeline.run(context="Data VisuliZation",transcripts=processed_transcripts_set)

> Running module 3b6c447c-0429-41db-9148-70689a0ea94a with input: 
context: Data VisuliZation
transcripts: ["[Music]let's now start this lesson by definingwhat data visualization isdata visualization is the technique topresent the data in a pictorial orgraphical formatit enables stakeholders and decisionma...

> Running module 16a38c67-3f84-48d0-8112-25392ae48a19 with input: 
messages: 
You are a Youtube Content Creation Subject Matter Expert.You need to create a
youtube video script on the givenData VisuliZation.you will be given  5 youtube video ["[Music]let's now start this lesso...



In [132]:
print(llm_response)

assistant: ```json
[
  {
    "timestamp": "0:00-2:00",
    "timeblock": "Introduction",
    "summary": "Start with a hook about the overwhelming amount of data in the world. Introduce data visualization as a solution. Briefly mention the benefits like pattern identification, storytelling, and improved decision-making.",
    "script": "Hey everyone, have you ever felt like you're drowning in a sea of data? We're bombarded with information every day, from news articles and social media feeds to complex spreadsheets at work. It's easy to feel overwhelmed and struggle to make sense of it all. But what if there was a way to cut through the noise and unlock the hidden stories within our data? That's where data visualization comes in. By transforming raw data into engaging visuals like charts, graphs, and maps, we can reveal patterns, identify trends, and communicate insights more effectively. In this video, we'll explore the power of data visualization and how it can help us make better deci